In [1]:
import requests
import bs4
import os
import re
import unicodedata
import pandas as pd
import nltk
import numpy as np

In [3]:
url = "https://www.ebay.com/sch/i.html?_from=R40&_nkw=rtx+3060&_sacat=175673&rt=nc&Chipset%252FGPU%2520Model=NVIDIA%2520GeForce%2520RTX%25203060%2520Ti%7CNVIDIA%2520GeForce%2520RTX%25203060&_dcat=27386&_ipg=200"
response = requests.get(url)
html = response.text
soup = bs4.BeautifulSoup(html)

In [4]:
response

<Response [200]>

In [8]:
items = soup.find_all('div', {'class': 's-item__info clearfix'})

len(items)

214

In [73]:
items[8].find('h3', {'class': "s-item__title"}).text

'New ListingEVGA GeForce RTX 3060 Ti XC GAMING 8GB GDDR6 Graphics Card (08G-P5-3663-KL)'

In [72]:
items[8].find('span', {'class': 'SECONDARY_INFO'}).text

'Brand New'

In [71]:
items[8].find('span', {'class': 's-item__price'}).text

'$660.00'

In [63]:
items[8].find('span', {'class': 's-item__bids s-item__bidCount'}).text

'29 bids'

In [79]:
items[8].find('span', {'class': 's-item__time-left',}).text

'6d 20h left'

In [81]:
items[8].find('span', {'class': 's-item__shipping s-item__logisticsCost'}).text

'+$20.00 shipping'

In [ ]:
<span class="s-item__shipping s-item__logisticsCost">Free shipping</span>


In [ ]:
for item in items:
    
    title = item.find('h3', {'class': "s-item__title"}).text
    condition = item.find('span', {'class': 'SECONDARY_INFO'}).text
    current_price = item.find('span', {'class': 's-item__price'}).text
    total_bids = item.find('span', {'class': 's-item__bids s-item__bidCount'}).text
    bidding_time_left = item.find('span', {'class': 's-item__time-left',}).text
    shipping_cost = item.find('span', {'class': 's-item__shipping s-item__logisticsCost'}).text
    
    if type(total_bids) is None:
        total_bids = np.nan
    else:
        total_bids = total_bids
        
    if type(bidding_time_left) is None:
        bidding_time_left = np.nan
    else:
        bidding_time_left = bidding_time_left
    
    